<a href="https://colab.research.google.com/github/raulrosapacheco/Dados-Telecom-Analise-Limpeza/blob/main/Dados_Telecom_Analise_Limpeza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise e Limpeza de Dados de Telecomunicações

Fonte de Dados: <a href ="https://www.kaggle.com/datasets/blastchar/telco-customer-churn">Telco Customer Churn<a>

Uma empresa de telecomunicações possui um grande volume de dados de clientes.

Neste projeto daremos destaque ao tratamento de valores ausentes, tratamento de outliers e ajuste dos tipos de dados usando linguagem Python.

## Instalando e Carregando Pacotes

In [1]:
# Imports
import math
import sys, os
import numpy as np
import pandas as pd

In [2]:
# Import python modules from github
#!wget -q https://raw.githubusercontent.com/raulrosapacheco/Dados-Telecom-Analise-Limpeza/main/modulos/estrategia1.py
#!wget -q https://raw.githubusercontent.com/raulrosapacheco/Dados-Telecom-Analise-Limpeza/main/modulos/estrategia2.py
#!wget -q https://raw.githubusercontent.com/raulrosapacheco/Dados-Telecom-Analise-Limpeza/main/modulos/estrategia3.py

In [3]:
# Config Pandas
pd.set_option('display.max_columns', 100)

## Carregando Dados

In [4]:
# Load Dataset
list_missing_values = ["n/a", "na", "undefined"]
dataset = pd.read_csv("https://raw.githubusercontent.com/raulrosapacheco/Dados-Telecom-Analise-Limpeza/main/dados/dataset.csv", na_values = list_missing_values)
dataset.shape

(150001, 55)

In [5]:
dataset.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),HTTP DL (Bytes),HTTP UL (Bytes),Activity Duration DL (ms),Activity Duration UL (ms),Dur. (ms).1,Handset Manufacturer,Handset Type,Nb of sec with 125000B < Vol DL,Nb of sec with 1250B < Vol UL < 6250B,Nb of sec with 31250B < Vol DL < 125000B,Nb of sec with 37500B < Vol UL,Nb of sec with 6250B < Vol DL < 31250B,Nb of sec with 6250B < Vol UL < 37500B,Nb of sec with Vol DL < 6250B,Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.0,5.0,23.0,44.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,37624.0,38787.0,1.823653e+09,Samsung,Samsung Galaxy A5 Sm-A520F,NaN,NaN,NaN,NaN,NaN,NaN,213.0,214.0,1545765.0,24420.0,1634479.0,1271433.0,3563542.0,137762.0,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.0,5.0,16.0,26.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,168.0,3560.0,1.365104e+09,Samsung,Samsung Galaxy J5 (Sm-J530),NaN,NaN,NaN,NaN,NaN,NaN,971.0,1022.0,1926113.0,7165.0,3493924.0,920172.0,629046.0,308339.0,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,NaN,NaN,6.0,9.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.361763e+09,Samsung,Samsung Galaxy A8 (2018),NaN,NaN,NaN,NaN,NaN,NaN,751.0,695.0,1684053.0,42224.0,8535055.0,1694064.0,2690151.0,672973.0,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,NaN,NaN,44.0,44.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,3330.0,37882.0,1.321510e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,207.0,644121.0,13372.0,9023734.0,2788027.0,1439754.0,631229.0,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,NaN,NaN,6.0,9.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.089009e+09,Samsung,Samsung Sm-G390F,NaN,NaN,NaN,NaN,NaN,NaN,607.0,604.0,862600.0,50188.0,6248284.0,1500559.0,1936496.0,173853.0,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [6]:
# Load data dictionary
dictionary = pd.read_excel("https://github.com/raulrosapacheco/Dados-Telecom-Analise-Limpeza/blob/main/dados/Dicionario.xlsx?raw=true")
dictionary.shape

(56, 2)

In [7]:
dictionary.head()

,Fields,Description
0,bearer id,xDr session identifier
1,Dur. (ms),Total Duration of the xDR (in ms)
2,Start,Start time of the xDR (first frame timestamp)
3,Start ms,Milliseconds offset of start time for the xDR ...
4,End,End time of the xDR (last frame timestamp)


## Análise Exploratória

In [8]:
# Info
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [9]:
# Statistical description
dataset.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),HTTP DL (Bytes),HTTP UL (Bytes),Activity Duration DL (ms),Activity Duration UL (ms),Dur. (ms).1,Nb of sec with 125000B < Vol DL,Nb of sec with 1250B < Vol UL < 6250B,Nb of sec with 31250B < Vol DL < 125000B,Nb of sec with 37500B < Vol UL,Nb of sec with 6250B < Vol DL < 31250B,Nb of sec with 6250B < Vol UL < 37500B,Nb of sec with Vol DL < 6250B,Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,150000.000000,6.185500e+04,5.335200e+04,149247.000000,149247.000000,149247.000000,149247.000000,149209.000000,149209.000000,149209.000000,149209.000000,6.852700e+04,6.819100e+04,1.500000e+05,1.500000e+05,1.500000e+05,52463.000000,57107.000000,56415.000000,19747.000000,61684.000000,38158.000000,149246.000000,149208.000000,1.500010e+05,150001.000000,1.500010e+05,1.500010e+05,1.500010e+05,150001.000000,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,1770.428647,2.080991e+07,7.596587e+05,92.844754,3.069355,1.717341,1.609654,98.530142,0.776749,0.147987,0.078923,1.144710e+08,3.242301e+06,1.829177e+06,1.408880e+06,1.046091e+08,989.699998,340.434395,810.837401,149.257052,965.464756,141.304812,3719.787552,4022.083454,1.795322e+06,32928.434380,5.750753e+06,2.056542e+06,1.791729e+06,467373.441940,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,4625.355500,1.825665e+08,2.645305e+07,13.038031,6.215233,4.159538,4.828890,4.634285,3.225176,1.624523,1.295396,9.631946e+08,1.957064e+07,5.696395e+06,4.643231e+06,8.103761e+07,2546.524440,1445.365032,1842.162008,1219.112287,1946.387608,993.349688,9171.609010,10160.324314,1.035482e+06,19006.178256,3.309097e+06,1.189917e+06,1.035840e+06,269969.307031,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,0.000000,2.000000e+00,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000e+01,4.000000e+01,0.000000e+00,0.000000e+00,7.142988e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.200000e+01,0.000000,2.070000e+02,3.000000e+00,1.400000e+01,2.000000,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,47.000000,3.565150e+04,4.694750e+03,91.000000,0.000000,0.000000,0.000000,99.000000,0.000000,0.000000,0.000000,1.124035e+05,2.432200e+04,1.487775e+04,2.153975e+04,5.744079e+07,20.000000,10.000000,26.000000,2.000000,39.000000,3.000000,87.000000,106.000000,8.991480e+05,16448.000000,2.882393e+06,1.024279e+06,8.927930e+05,233383.000000,5.833501e+06,5.517965e+06,5.777156

* **Bearer Id:** Identificador de sessão
* **IMSI:** Identidade do Assinante Móvel Internacional
* **MSISDN/Number:** Número do Celular
* **IMEI:** Identidade Internacional de Equipamento Móvel

As variáveis Beared Id, IMSI, MSISDN/Number e IMEI foram tipificadas pela linguagem Python como variáveis numéricas, porém não faz sentido calcular estatísticas descritivas destas.

In [10]:
# Shape
print(dataset.shape)
print(dictionary.shape)

(150001, 55)
(56, 2)


Há mais colunas descritas no dicionário de dados do que realmente existem no DataFrame.

In [11]:
# Compare dictionary columns with existing ones in dataframe
df_compare_columns = pd.concat([pd.Series(dataset.columns.tolist()), dictionary['Fields']], axis = 1)
df_compare_columns.rename(columns = {0: 'Dataset Column', 'Fields': 'Dictionary Columns'}, inplace = True)
df_compare_columns

,Dataset Column,Dictionary Columns
0,Bearer Id,bearer id
1,Start,Dur. (ms)
2,Start ms,Start
3,End,Start ms
4,End ms,End
5,Dur. (ms),End ms
6,IMSI,Dur. (s)
7,MSISDN/Number,IMSI
8,IMEI,MSISDN/Number
9,Last Location Name,IMEI


No dicionário de dados, a variável representada com o de índice 2 'Dur.(ms)' não corresponde a corresponde a variável de indice 2 do dataset. 

As colunas de indice 5 e 28 do dataset parecem estar duplicadas, é preciso verificar.

In [12]:
dataset[['Dur. (ms)', 'Dur. (ms).1']]

,Dur. (ms),Dur. (ms).1
0,1823652.0,1.823653e+09
1,1365104.0,1.365104e+09
2,1361762.0,1.361763e+09
3,1321509.0,1.321510e+09
4,1089009.0,1.089009e+09
...,...,...
149996,81230.0,8.123076e+07
149997,97970.0,9.797070e+07
149998,98249.0,9.824953e+07
149999,97910.0,9.791063e+07


A coluna Dur. (ms) é medida em segundos, é preciso renomea-la. 

As colunas 'Dur. (ms).1', 'Start ms', 'End ms' também serão renomeadas para manter o padrão de nomenclatura do Dataset.

In [13]:
dataset.rename(columns = {'Dur. (ms)': 'Dur (s)', 
                          'Dur. (ms).1': 'Dur (ms)', 
                          'Start ms': 'Start Offset (ms)', 
                          'End ms': 'End Offset (ms)'}, 
               inplace = True)

dataset.columns.tolist()

['Bearer Id',
 'Start',
 'Start Offset (ms)',
 'End',
 'End Offset (ms)',
 'Dur (s)',
 'IMSI',
 'MSISDN/Number',
 'IMEI',
 'Last Location Name',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'HTTP DL (Bytes)',
 'HTTP UL (Bytes)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur (ms)',
 'Handset Manufacturer',
 'Handset Type',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 12

## Tratamento de Valores Ausentes

In [14]:
# Function: Calculate Percent Missing Values in DataFrame
def func_percent_values_missing(df):
  totalCells = np.product(df.shape)
  missingCount = df.isnull().sum()
  totalMissing = missingCount.sum()
  print("O dataset tem", round(((totalMissing/totalCells) * 100), 2), "% de valores ausentes")

In [15]:
# Function: Calculate Percent Missing Values by Column
def func_percent_values_missing_column(df):
  missingCount = df.isnull().sum()
  missingPercent = 100 * missingCount / len(df)
  missingType = df.dtypes
  missingTable = pd.concat([missingCount, missingPercent, missingType], axis = 1)
  mis_table_ren_columns = missingTable.rename(columns = {0: 'Valores Ausentes', 1: '% de Valores Ausentes', 2: 'Dtype'})
  mis_table_ren_columns = mis_table_ren_columns[mis_table_ren_columns.iloc[:,0] != 0].sort_values('% de Valores Ausentes', ascending = False).round(2)
  print("O dataset tem " + str(df.shape[1]) + " colunas. \nEncontrando: " + str(mis_table_ren_columns.shape[0]) + " colunas que têm valores ausentes.")
  if mis_table_ren_columns.shape[0] == 0:
    return
  return mis_table_ren_columns

In [31]:
# Fuction: Calculate Percent of rows with Missing Values
def func_percent_values_missing_row(df):
  missing_rows = sum([True for idx,row in df.iterrows() if any(row.isna())])
  total_rows = df.shape[0]
  print(round(((missing_rows/total_rows) * 100), 2), "% das linhas no conjunto de dados contêm pelo menos um valor ausente.")

In [16]:
# Function: Imputation with forward fill
def fix_missing_ffill(df, col):
  count = df[col].isna().sum()
  df[col] = df[col].fillna(method = 'ffill')
  print(f"{count} valores ausentes na {col} foram substituídos usando método de preenchimento reverso.")
  return df[col]

In [17]:
# Function: Imputation with backward fill
def fix_missing_bfill(df, col):
    count = df[col].isna().sum()
    df[col] = df[col].fillna(method = 'bfill')
    print(f"{count} valores ausentes na coluna {col} foram substituídos usando o método de preenchimento reverso.")
    return df[col]

In [37]:
# Function: Fill missing values with something
def fix_missing_value(df, col, value):
    count = df[col].isna().sum()
    df[col] = df[col].fillna(value)
    if type(value) == 'str':
        print(f"{count} valores ausentes na coluna {col} foram substituídos por '{value}'.")
    else:
        print(f"{count} valores ausentes na coluna {col} foram substituídos por {value}.")
    return df[col]

In [40]:
# Function: Drop Rows with Values Ausentes
def drop_rows_with_missing_values(df):
    old = df.shape[0]
    df.dropna(inplace = True)
    new = df.shape[0]
    count = old - new
    print(f"{count} linhas contendo valores ausentes foram descartadas.")

In [36]:
# Calculate Percent Missing Values in 'datset'
func_percent_values_missing(dataset)

O dataset tem 12.72 % de valores ausentes


In [19]:
# Create table of missing values by Column
df_missing = func_percent_values_missing_column(dataset)
df_missing

O dataset tem 55 colunas. 
Encontrando: 41 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
Nb of sec with 37500B < Vol UL,130254,86.84,float64
Nb of sec with 6250B < Vol UL < 37500B,111843,74.56,float64
Nb of sec with 125000B < Vol DL,97538,65.02,float64
TCP UL Retrans. Vol (Bytes),96649,64.43,float64
Nb of sec with 31250B < Vol DL < 125000B,93586,62.39,float64
Nb of sec with 1250B < Vol UL < 6250B,92894,61.93,float64
Nb of sec with 6250B < Vol DL < 31250B,88317,58.88,float64
TCP DL Retrans. Vol (Bytes),88146,58.76,float64
HTTP UL (Bytes),81810,54.54,float64
HTTP DL (Bytes),81474,54.32,float64


Em análise de dados, é comum fazer a remoção das colunas com mais de 30% de valores ausentes. Porém é preciso alinhar os conhecimentos de negócio para tomar as melhores decisões.

In [20]:
# List of columns to be removed
columns_to_remove = df_missing[df_missing['% de Valores Ausentes'] >= 30.00].index.tolist()
columns_to_remove

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'TCP UL Retrans. Vol (Bytes)',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'TCP DL Retrans. Vol (Bytes)',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

* **TCP UL Retrans. Vol (Bytes):** Volume TCP de pacotes Uplink detectados como retransmitidos (bytes)
* **TCP DL Retrans. Vol (Bytes):** Volume TCP de pacotes Downlink detectados como retransmitidos (bytes) 

Por decisão de negócio, as variáveis TCP passãrão por imputação, pois estas podem ser necessárias para análises futuras.

In [21]:
# Columns to be removed
columns_to_remove = [col for col in columns_to_remove if col not in ['TCP UL Retrans. Vol (Bytes)', 'TCP DL Retrans. Vol (Bytes)']]
columns_to_remove

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

In [22]:
# Drop columns
dataset_clean = dataset.drop(columns_to_remove, axis = 1)
dataset_clean.shape

(150001, 47)

In [23]:
# Calculate Percent Missing Values 
func_percent_values_missing(dataset_clean)

O dataset tem 3.85 % de valores ausentes


In [24]:
# Calculate Percent Missing Values by column
func_percent_values_missing_column(dataset_clean)

O dataset tem 47 colunas. 
Encontrando: 33 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
TCP UL Retrans. Vol (Bytes),96649,64.43,float64
TCP DL Retrans. Vol (Bytes),88146,58.76,float64
Avg RTT DL (ms),27829,18.55,float64
Avg RTT UL (ms),27812,18.54,float64
Handset Type,9559,6.37,object
Handset Manufacturer,9559,6.37,object
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float64
Bearer Id,991,0.66,float64
Nb of sec with Vol UL < 1250B,793,0.53,float64


A porcentagem de valores ausentes das colunas '**TCP UL Retrans. Vol (Bytes)**' e '**TCP DL Retrans. Vol (Bytes)**' são muito altos (>30%), portanto um único valor como média ou mediana não é aconselhável. Para estas variáveis será usado o método de preenchimento reverso.


In [25]:
# Imputation with backward fill in column 'TCP UL Retrans. Vol (Bytes)'
fix_missing_bfill(dataset_clean, 'TCP UL Retrans. Vol (Bytes)')

96649 valores ausentes na coluna TCP UL Retrans. Vol (Bytes) foram substituídos usando o método de preenchimento reverso.


0         7230.0
1         7230.0
2         7230.0
3         7230.0
4         7230.0
           ...  
149996       NaN
149997       NaN
149998       NaN
149999       NaN
150000       NaN
Name: TCP UL Retrans. Vol (Bytes), Length: 150001, dtype: float64

In [26]:
# Imputation with backward fill in column 'TCP UL Retrans. Vol (Bytes)'
fix_missing_bfill(dataset_clean, 'TCP DL Retrans. Vol (Bytes)')

88146 valores ausentes na coluna TCP DL Retrans. Vol (Bytes) foram substituídos usando o método de preenchimento reverso.


0         19520.0
1         19520.0
2         19520.0
3         19520.0
4         19520.0
           ...   
149996        NaN
149997        NaN
149998        NaN
149999        NaN
150000        NaN
Name: TCP DL Retrans. Vol (Bytes), Length: 150001, dtype: float64

As variáves '**Avg RTT DL (ms)**' e '**Avg RTT UL (ms)**' têm as próximas porcentagens mais altas de valores ausentes com cerca de 18,5% cada. 
Para utilizar imputação com a média ou mediana é preciso verificar se as variáveis seguem uma distribuição normal.

O coeficiente de assimetria mede a distribução dos dados em torno da média: 
* Se a assimetria estiver entre -0,5 e 0,5, os dados são bastante simétricos;
* Se a assimetria estiver entre -1 e - 0,5 ou entre 0,5 e 1, os dados estão moderadamente inclinados;
* Se a assimetria for menor que -1 ou maior que 1, os dados estão altamente enviesados.

In [27]:
# Calculation of asymmetry coefficient for variable 'Avg RTT DL (ms)'
dataset_clean['Avg RTT DL (ms)'].skew(skipna = True)

62.90782807995961

In [28]:
# Calculation of asymmetry coefficient for variable 'Avg RTT UL (ms)'
dataset_clean['Avg RTT UL (ms)'].skew(skipna = True)

28.45741458546382

As variáves '**Avg RTT DL (ms)**' e '**Avg RTT UL (ms)**' são extremamentes enviesadas positivamente. Portanto será usado o método de preenchimento progressivo.

In [29]:
# Imputation with forward fill in column 'Avg RTT DL (ms)'
fix_missing_ffill(dataset_clean, 'Avg RTT DL (ms)')

27829 valores ausentes na Avg RTT DL (ms) foram substituídos usando método de preenchimento reverso.


0         42.0
1         65.0
2         65.0
3         65.0
4         65.0
          ... 
149996    32.0
149997    27.0
149998    43.0
149999    37.0
150000    37.0
Name: Avg RTT DL (ms), Length: 150001, dtype: float64

In [30]:
# Imputation with forward fill in column 'Avg RTT UL (ms)'
fix_missing_ffill(dataset_clean, 'Avg RTT UL (ms)')

27812 valores ausentes na Avg RTT UL (ms) foram substituídos usando método de preenchimento reverso.


0         5.0
1         5.0
2         5.0
3         5.0
4         5.0
         ... 
149996    0.0
149997    2.0
149998    6.0
149999    5.0
150000    5.0
Name: Avg RTT UL (ms), Length: 150001, dtype: float64

In [33]:
# Percent Missing Values in 'data_clean'
func_percent_values_missing(dataset_clean)

O dataset tem 0.44 % de valores ausentes


In [34]:
# Percent Rows with Missing Values
func_percent_values_missing_row(dataset_clean)

7.96 % das linhas no conjunto de dados contêm pelo menos um valor ausente.


In [35]:
# Percent Missing Values by Column
func_percent_values_missing_column(dataset_clean)

O dataset tem 47 colunas. 
Encontrando: 31 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
Handset Type,9559,6.37,object
Handset Manufacturer,9559,6.37,object
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float64
Bearer Id,991,0.66,float64
Nb of sec with Vol UL < 1250B,793,0.53,float64
UL TP > 300 Kbps (%),792,0.53,float64
50 Kbps < UL TP < 300 Kbps (%),792,0.53,float64
10 Kbps < UL TP < 50 Kbps (%),792,0.53,float64
UL TP < 10 Kbps (%),792,0.53,float64


"**Handset Type**" e "**Handset Manufacturer**" são colunas categóricas, é melhor imputá-los com o valor "unknown" para que não enviesemos os dados.

In [38]:
# Imputation of categorical variables
fix_missing_value(dataset_clean, 'Handset Type', 'unknown')
fix_missing_value(dataset_clean, 'Handset Manufacturer', 'unknown')

9559 valores ausentes na coluna Handset Type foram substituídos por unknown.
9559 valores ausentes na coluna Handset Manufacturer foram substituídos por unknown.


0         Samsung
1         Samsung
2         Samsung
3         unknown
4         Samsung
           ...   
149996      Apple
149997      Apple
149998      Apple
149999     Huawei
150000    unknown
Name: Handset Manufacturer, Length: 150001, dtype: object

In [39]:
# Percent Rows with Missing Values
func_percent_values_missing_row(dataset_clean)

2.08 % das linhas no conjunto de dados contêm pelo menos um valor ausente.


Uma vez que apenas 2,08% das linhas contêm pelo menos um valor ausente e o número total de linhas é de cerca de 150000, descartar essas linhas não terá um impacto negativo perceptível. 5% ou menos, normalmente descartamos.

In [41]:
# Drop Rows with Values Ausentes
drop_rows_with_missing_values(dataset_clean)

3114 linhas contendo valores ausentes foram descartadas.


In [45]:
# Percent Missing Values in 'dataset_clean'
func_percent_values_missing(dataset_clean)
print('Formato:', dataset_clean.shape)

O dataset tem 0.0 % de valores ausentes
Formato: (146887, 47)


## Conversão de Tipos de Dados